In [4]:
import polars as pl
from deltalake.writer import write_deltalake
import os
from datetime import date 
from pathlib import Path
import shutil
import stat

In [5]:
# VARIAVEIS E FUNCOES

# caminho dos arquivos csv's de movimentação
path_movimetacao = Path("C:/Users/dougl/OneDrive/Área de Trabalho/ESTUDOS/nubank-data-engineering-portfolio/arquivos_nubank/conta")

# caminho dos arquivos csv's de gastos em créditos
path_credito = Path("C:/Users/dougl/OneDrive/Área de Trabalho/ESTUDOS/nubank-data-engineering-portfolio/arquivos_nubank/cartao_credito")

# data atual para ser utilizada como parte do nome das pastas em que será salvo os arquivos parquet
today = date.today().isoformat()

# caminho para salvar arquivo parquet final na pasta bronze
path_bronze = Path(f"C:/Users/dougl/OneDrive/Área de Trabalho/ESTUDOS/nubank-data-engineering-portfolio/data/bronze/{today}")

# Remove a pasta e arquivos dentro dela caso já exista (path_bronze)
def remove_readonly(func, path, _):
    """Altera permissão para escrita e tenta remover o arquivo novamente"""
    os.chmod(path, stat.S_IWRITE)
    func(path)

In [6]:
# PREPARAR PASTAS PARA RECEBER DELTAS COM DADOS INGERIDOS DOS CSV'S

# Excluir a pasta se ela já existir
if os.path.exists(path_bronze):
    shutil.rmtree(path_bronze, onerror=remove_readonly)

# criar o caminho para salvar os arquivos
os.makedirs(path_bronze)

In [7]:
## Função para unificar os arquivos e retornar em um dataframe para que possa salvar na camada raw

def unifica_arquivos(local_arquivo):
    lista_dfs = []
        
    for arquivo in os.listdir(local_arquivo):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(local_arquivo, arquivo)
            df = pl.read_csv(caminho_arquivo)
            lista_dfs.append(df)

    arquivo_final = pl.concat(lista_dfs, how = 'vertical')

    return arquivo_final

In [ ]:
# ingestão dos dados dos csv's de movimentação da conta

df_movimentacao = unifica_arquivos(path_movimetacao)

caminho_bronze_movimentacao = Path(path_bronze/"movimentacao")

write_deltalake(caminho_bronze_movimentacao, df_movimentacao, mode = 'overwrite')

In [ ]:
# ingestão dos dados dos csv's de compras em crédito

df_credito = unifica_arquivos(path_credito)

caminho_bronze_credito = Path(path_bronze/"credito")

write_deltalake(caminho_bronze_credito, df_credito, mode = 'overwrite')

In [10]:
display(df_movimentacao)

Data,Valor,Identificador,Descrição
str,f64,str,str
"""05/04/2023""",8850.95,"""767e15ff-21b2-4f53-b38b-c5a06d…","""Transferência recebida pelo Pi…"
"""07/04/2023""",-385.92,"""d1957c41-bdf4-4334-a51f-5d28b0…","""Compra no débito - Assaí"""
"""08/04/2023""",-2086.44,"""79c0eb87-af0c-4fc7-acc8-7b9361…","""Pagamento de boleto - Aluguel"""
"""08/04/2023""",-10.64,"""60a56b98-6a8d-41ff-81d5-57122f…","""Compra no débito - Café Centra…"
"""09/04/2023""",-561.15,"""d90633b5-e73d-4f18-98ac-4649b5…","""Transferência enviada pelo Pix…"
…,…,…,…
"""19/09/2024""",-172.91,"""3b6cfdd8-2f46-42f8-8999-29eaad…","""Pagamento de boleto - Água Sab…"
"""19/09/2024""",-2372.26,"""5bf492d9-f561-4ba1-bbaa-20a030…","""Pagamento fatura cartão Nubank"""
"""19/09/2024""",-106.53,"""e09196e4-8ac8-4ba7-9f25-9ffff3…","""Pagamento de boleto - Escola"""
